In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

In [22]:
cin = 3 # input channel size
cout = 4 # output channel size
k_size = 2 # kernel size
stride = 1
padding = 1
dilation = 2

# create_data
torch.manual_seed(42)
n = 2
lin = 4
x = torch.randn(n,cin,lin)

# convolution with module
conv = nn.Conv1d(cin, cout ,k_size, stride, padding, dilation=dilation)
linear = nn.Linear(5,3)
mout = conv(x)

# --------------- scratch  --------------
# padding
x_padded = F.pad(x, (padding, padding), 'constant', 0)
# convolution with scratch
lout = (lin + 2*padding - dilation*(k_size-1) - 1)//stride + 1
sout = torch.zeros(n, cout, lout)
for i in range(n):
    for j in range(cout):
        for l in range(lout):
            trg = x_padded[i, :, l:l + k_size + dilation - 1:dilation]
            sout[i,j,l] = torch.sum(trg*conv.weight[j]) + conv.bias[j]

print('-------------------------------with_scratch-------------------------\n', sout)
print()
print('-------------------------------with_module--------------------------\n', mout)
print()
print('----------------------------------valid-----------------------------\n',
torch.round(mout, decimals=5)==torch.round(sout, decimals=5))

-------------------------------with_scratch-------------------------
 tensor([[[-0.3670,  0.5096, -0.4808,  0.1001],
         [ 0.4391,  0.5202,  0.4944,  0.3177],
         [ 0.6329,  0.1589, -1.3393, -0.0122],
         [-0.2328,  0.2560,  1.1390, -0.0174]],

        [[-0.0180,  0.5075, -0.2094, -0.1865],
         [ 0.0300, -0.4103, -0.0334, -0.1297],
         [ 0.3028, -0.0608,  0.0752,  1.0516],
         [-0.1494, -0.1147, -0.0413, -1.0659]]], grad_fn=<CopySlices>)

-------------------------------with_module--------------------------
 tensor([[[-0.3670,  0.5096, -0.4808,  0.1001],
         [ 0.4391,  0.5202,  0.4944,  0.3177],
         [ 0.6329,  0.1589, -1.3393, -0.0122],
         [-0.2328,  0.2560,  1.1390, -0.0174]],

        [[-0.0180,  0.5075, -0.2094, -0.1865],
         [ 0.0300, -0.4103, -0.0334, -0.1297],
         [ 0.3028, -0.0608,  0.0752,  1.0516],
         [-0.1494, -0.1147, -0.0413, -1.0659]]],
       grad_fn=<ConvolutionBackward0>)

----------------------------------val

In [ ]:
# backward


In [5]:
import numpy as np

In [6]:
class Conv1d():
    def __init__(self, cin: int, cout: int, k_size: int,
                    stride: int, padding: int, dilation: int):
        """
        cin : input_channel_size_
        cout : output_channel_size_
        k_size : kernel_size_
        """
        self.cin = cin
        self.cout = cout
        self.k_size = k_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        
        self.weight = np.random.randn(cout, cin, k_size)
        self.bias = np.random.randn(cout)

    def __call__(self, x: np.ndarray) -> np.ndarray: 
        """ forward_propagation
        x : input_data_ (batch_size, cin, lin)
        out : output_data_ (batch_size, cout, lout)
        """
        # padding
        x_padded = np.pad(x, [(0,0),(0,0),(self.padding, self.padding)], 'constant')
        self.x_padded = x_padded
        # convolution
        lout = (lin + 2*padding - dilation*(self.k_size-1) - 1)//stride + 1
        output = torch.zeros(n, cout, lout)
        for i in range(n):
            for j in range(cout):
                for l in range(lout):
                    trg = x_padded[i, :, l:l + self.k_size + dilation - 1:dilation]
                    output[i,j,l] = torch.sum(trg*conv.weight[j]) + conv.bias[j]
        return output

    def backward(self, grad: np.ndarray) -> np.ndarray:
        grad = self.x_padded * grad
        return 

In [40]:
np.pad(x, [(0,0),(0,0),(2,1)], 'constant')

array([[[ 0.        ,  0.        ,  1.926915  ,  1.4872842 ,
          0.9007172 , -2.1055214 ,  0.        ],
        [ 0.        ,  0.        ,  0.67841846, -1.234545  ,
         -0.04306748, -1.604667  ,  0.        ],
        [ 0.        ,  0.        ,  0.3558598 , -0.686623  ,
         -0.49335635,  0.2414878 ,  0.        ]],

       [[ 0.        ,  0.        , -1.1109041 ,  0.09154566,
         -2.3169227 , -0.21680473,  0.        ],
        [ 0.        ,  0.        , -0.30972692, -0.3957105 ,
          0.80340934, -0.6215954 ,  0.        ],
        [ 0.        ,  0.        , -0.5920003 , -0.06307438,
         -0.8285543 ,  0.33089843,  0.        ]]], dtype=float32)